In [ ]:
! git clone https://github.com/qwoprocks/EfficientDet.git

import os
import sys

sys.path.append("/kaggle/working/EfficientDet")
sys.path.append("/kaggle/working/EfficientDet/utils")

%cd ./EfficientDet
! python setup.py build_ext --inplace

open('__init__.py', 'a').close()

In [ ]:
from model import efficientdet

import os, csv, math, shutil, zipfile, urllib
import xml.etree.ElementTree as ET
import numpy as np
import pandas

annotations = []
fullvalannotations = []

#FOR ANNOTATIONS
with open('/kaggle/input/retinanetcsvfiles/training.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        item = ['/kaggle/input/til2020/'+row['filename']] + [math.floor(float(row['xmin'])),math.floor(float(row['ymin'])),math.floor(float(row['xmax'])),math.floor(float(row['ymax']))] + [row['class']]
        annotations.append(item)

#FOR FULL VALIDATION ANNOTATIONS
with open('/kaggle/input/retinanetcsvfiles/validation.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        item = ['/kaggle/input/til2020/'+row['filename']] + [math.floor(float(row['xmin'])),math.floor(float(row['ymin'])),math.floor(float(row['xmax'])),math.floor(float(row['ymax']))] + [row['class']]
        fullvalannotations.append(item)

In [ ]:
classes = ['tops','trousers','outerwear','dresses','skirts']
for i, line in enumerate(classes):
    print('{},{}'.format(line,i))

In [ ]:
ANNOTATIONS_FILE = '/kaggle/working/annotations.csv'
FULLVALANNOTATIONS_FILE = '/kaggle/working/fullvalannotations.csv'
CLASSES_FILE = '/kaggle/working/classes.csv'

with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)

with open(CLASSES_FILE, 'w') as f:
  writer = csv.writer(f)
  for i, line in enumerate(classes):
    f.write('{},{}\n'.format(line,i))
  
with open(FULLVALANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(fullvalannotations)

## Training

In [ ]:
TRAIN = "/kaggle/working/EfficientDet/train.py"

! python3 {TRAIN} --snapshot imagenet --phi 4 --weighted-bifpn --snapshot-path "/kaggle/working" --freeze-backbone --gpu 0 --random-transform --multiprocessing --batch-size 4 --workers 8 --steps 1000 --epochs 10 csv {ANNOTATIONS_FILE} {CLASSES_FILE}

## Evaluation

In [ ]:
import glob

list_of_files = glob.glob('/kaggle/working/*.h5')
list_of_files.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))

### LOWEST TRAINING LOSS
best_model = list_of_files[0]
print(best_model)

In [ ]:
_, prediction_model = efficientdet(4,
                               num_classes=5,
                               num_anchors=9,
                               freeze_bn=True,
                               weighted_bifpn=True,
                               )
prediction_model.load_weights(best_model, by_name=True)

In [ ]:
from tqdm.notebook import tqdm
import cv2
import cupy as cp

def preprocess_image_gpu(image):
    # image, RGB
    image_size = 1408
    image_height, image_width = image.shape[:2]
    if image_height > image_width:
        scale = image_size / image_height
        resized_height = image_size
        resized_width = int(image_width * scale)
    else:
        scale = image_size / image_width
        resized_height = int(image_height * scale)
        resized_width = image_size

    image = cv2.resize(image, (resized_width, resized_height))

    image = cp.array(image) #Numpy to Cupy
    image = image.astype(np.float32)
    image /= 255.
    mean = cp.array([0.485, 0.456, 0.406])
    std = cp.array([0.229, 0.224, 0.225])
    image -= mean
    image /= std

    newimg = cp.zeros((image_size,image_size,3),dtype=cp.float32)
    newimg[:image.shape[0],:image.shape[1]:,:] = image
    newimg = cp.asnumpy(newimg) #Cupy to Numpy

    return newimg, scale

detections = []
score_threshold = 0.01
max_detections = 100

for root, dirs, files in os.walk('/kaggle/input/til2020/val/val'):
    for filename in tqdm(files):
        img = '/kaggle/input/til2020/val/val/'+ filename
        image = cv2.imread(img)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:2]
        # preprocess image for network
        image, scale = preprocess_image_gpu(image)

        # process image
        boxes, scores, *_, labels = prediction_model.predict_on_batch([np.expand_dims(image, axis=0)])
        boxes /= scale
        boxes[:, :, 0] = np.clip(boxes[:, :, 0], 0, w - 1)
        boxes[:, :, 1] = np.clip(boxes[:, :, 1], 0, h - 1)
        boxes[:, :, 2] = np.clip(boxes[:, :, 2], 0, w - 1)
        boxes[:, :, 3] = np.clip(boxes[:, :, 3], 0, h - 1)
        
        # change to (x, y, w, h) (MS COCO standard)
        boxes[:, :, 2] -= boxes[:, :, 0]
        boxes[:, :, 3] -= boxes[:, :, 1]

        # select indices which have a score above the threshold
        indices = np.where(scores[0, :] > score_threshold)[0]
        scores = scores[0][indices]
        scores_sort = np.argsort(-scores)[:max_detections]
        
        image_boxes = boxes[0, indices[scores_sort], :]
        image_scores = scores[scores_sort]
        image_labels = labels[0, indices[scores_sort]]
        
        for box, score, label in zip(image_boxes, image_scores, image_labels):
          img_id = img.split("/")[-1]
          img_id = int(img_id.split(".")[0])
          cat_id = int(label) + 1
          conf = float(score)
          bbox = [round(elem, 1) for elem in box.tolist()]

          detections.append( {'image_id':img_id, 'category_id':cat_id, 'bbox':bbox, 'score':conf} )

print(detections[0])

In [ ]:
import json
with open('/kaggle/working/efficientdettop100.json', 'w') as f:
  json.dump(detections, f)

In [ ]:
! python -m pip install numpy==1.17.1
! pip install git+https://github.com/jinmingteo/cocoapi.git#subdirectory=PythonAPI

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

## Results

In [ ]:
coco_gt = COCO('/kaggle/input/til2020/val.json')
coco_dt = coco_gt.loadRes('/kaggle/working/efficientdettop100.json')
cocoEval = COCOeval(cocoGt=coco_gt, cocoDt=coco_dt, iouType='bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()